# Modelo de Mercado (Unifactorial)


El modelo de de mercado unifactorial consiste en tratar de explicar la rentabilidad de un activo a través del comportamiento del mercado, medido a través de la evolución de un indice. Es decir, se trata de realizar una regresión lineal unifactorial entre el comportamiento del indice y el comportamiento del mercado.

$$\tilde{R}_{it}=\alpha_i+\beta_i \tilde{R}_{Mt}+\tilde{\varepsilon}_{it}$$

La rentabilidad depende de tres componentes:
- $\alpha_i$: la parte del riesgo explicada por determinadas características del riesgo que no se deben a la evolución de la economía sino a características predecibles del título, se supone constante en el tiempo.
- $\beta_i$:la parte del riesgo que obedece a la evolución general de la economía que vienen recogido por el índice de mercado (RIESGO SISTEMÁTICO)
- $\varepsilon_{it}$:perturbación aleatoria del título, provocado por las decisiones o acontecimientos no predecibles (RIESGO ESPECÍFICO)

## Hipótesis del modelo

- El riesgo específico es una variable aleatoria con $E(\tilde{\varepsilon}_{it})=0$. Esto tiene sentido,cualquir componente predecible debe estar incluido en $\alpha_i$.
- El riesgo específico y el sistemático deben estar incorrelacionados $cov(R_{Mt},\varepsilon_{it}), \forall t$.
- No debe existir correlación entre los riesgos específicos de dos activos distintos $cov(\tilde{\varepsilon}_{it},\tilde{\varepsilon}_{jt}), \forall t$.
- No debe existir correlación entre los errores a lo largo del tiempo  $cov(\tilde{\varepsilon}_{it},\tilde{\varepsilon}_{jt'}),\forall t\neq t'$
- La varianza debe del riesgo específico ser constante a lo largo del tiempo $\sigma^2(\tilde{\varepsilon}_{it})=\sigma^2(\tilde{\varepsilon}_{it'}),\forall t\neq t'$.
- El riesgo sistemático se comporta como $\tilde{\varepsilon}_{it}=N(0,1),\forall t$

In [53]:
import pandas as pd
import yfinance as yf
import statsmodels.api as sm
import statsmodels.formula.api as smf
import numpy as np

In [44]:
RISKY_ASSET = 'AMZN'
MARKET_BENCHMARK = '^GSPC'
START_DATE = '2014-01-01'
END_DATE = '2018-12-31'

In [45]:
df = yf.download([RISKY_ASSET, MARKET_BENCHMARK],
                 start=START_DATE,
                 end=END_DATE,)['Adj Close']
df.head()

[*********************100%***********************]  2 of 2 completed


,AMZN,^GSPC
Date,,
2014-01-02,19.898500,1831.979980
2014-01-03,19.822001,1831.369995
2014-01-06,19.681499,1826.770020
2014-01-07,19.901501,1837.880005
2014-01-08,20.096001,1837.489990


Tenemos los datos diarios y queremos trabajar con datos mensuales. Para ello generamos un nuevo dataframe `X` y utilizamos la función `resample('M').last()` tomando como referencia el último día del mes con cotización.

In [46]:
df.index = pd.to_datetime(df.index)
X=df.resample('M').last()
X.head()

,AMZN,^GSPC
Date,,
2014-01-31,17.934500,1782.589966
2014-02-28,18.105000,1859.449951
2014-03-31,16.818501,1872.339966
2014-04-30,15.206500,1883.949951
2014-05-31,15.627500,1923.569946


In [47]:
monthly_ret = np.log(X).diff().dropna()
monthly_ret.head()

,AMZN,^GSPC
Date,,
2014-02-28,0.009462,0.042213
2014-03-31,-0.073709,0.006908
2014-04-30,-0.100757,0.006182
2014-05-31,0.027309,0.020812
2014-06-30,0.038384,0.018879


Calculamos la $\beta$ utilizando las varianza:
$$\beta=\frac{cov(AMZN,GSPC)}{\sigma^2(GSPC)}$$

In [48]:
covariance = monthly_ret.cov().iloc[0,1]
benchmark_variance = monthly_ret.cov().iloc[1,1]
beta = covariance / benchmark_variance
beta

1.6106258640720965

Estimamos la beta utilizando una regresión lineal para ello:
1. `y = monthly_ret.pop('AMZN')`: extraemos los datos de `AMZN`en una serie que denominamos `y`.
2. `monthly_ret=sm.add_constant(monthly_ret)`:añadimos una fila de unos en el dataframe `monthly_ret`.
3. `capm_model = sm.OLS(y, monthly_ret).fit()`: realizamos la regresión por mínimos cuadrados ordinarios.

In [49]:
y = monthly_ret.pop('AMZN')
y.head()

Date
2014-02-28    0.009462
2014-03-31   -0.073709
2014-04-30   -0.100757
2014-05-31    0.027309
2014-06-30    0.038384
Freq: M, Name: AMZN, dtype: float64

In [52]:
type(y)

pandas.core.series.Series

In [50]:
monthly_ret=sm.add_constant(monthly_ret)
monthly_ret.head()

,const,^GSPC
Date,,
2014-02-28,1.0,0.042213
2014-03-31,1.0,0.006908
2014-04-30,1.0,0.006182
2014-05-31,1.0,0.020812
2014-06-30,1.0,0.018879


In [51]:
capm_model = sm.OLS(y, monthly_ret).fit()
print(capm_model.summary())

                            OLS Regression Results                            
Dep. Variable:                   AMZN   R-squared:                       0.379
Model:                            OLS   Adj. R-squared:                  0.368
Method:                 Least Squares   F-statistic:                     34.78
Date:                Sun, 05 Nov 2023   Prob (F-statistic):           2.11e-07
Time:                        19:22:59   Log-Likelihood:                 77.379
No. Observations:                  59   AIC:                            -150.8
Df Residuals:                      57   BIC:                            -146.6
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0149      0.009      1.701      0.0

El riesgo total de un activo se puede descomponer en el riesgo sistemático y el específico:
$$\sigma^2_i=\beta_i\sigma^2_M+ \sigma^2_{\varepsilon_i}$$
Por tanto cuando diversiifcamos lo que estamos eliminando es el riesgo específico $\sigma^2_{\varepsilon_i}$.

## Consideramos el activo libre de riesgo

Una estrategia alternativa para estimar la beta es incorporar en la estimación el activo libre de riesgo. Si:
- $\tilde{\pi}_{it}=\tilde{R}_{it}-R_f$
- $\tilde{\pi}_{Mt}=\tilde{R}_{Mt}-R_f$

Por tanto planteamos el siguiente modelo. 

$$\tilde{\pi}_{it}=\alpha_i+\beta_i \tilde{pi}_{Mt}+\tilde{\varepsilon}_{it}$$

Utilizando datos del prof. Kenneth French: La prima de mercado $R_m-R_f$ y el tipo sin riesgo (aproximado por la letra del Tesoro a un mes) pueden descargarse del [sitio web](https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html) del profesor Kenneth French . Tenga en cuenta que la definición de referencia de mercado utilizada por el prof. French es diferente a la del índice S&P 500. En su sitio web encontrará una descripción detallada. 

Veamos como podemos trabajar con dichos datos:

1. Descargamos los datos: `!wget http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/F-F_Research_Data_Factors_CSV.zip`
2. Descomprimimos el archivo: `!unzip -a F-F_Research_Data_Factors_CSV.zip`. La opción `-a` asegura que los archivos de texto extraídos tengan el formato de fin de línea correcto para el sistema operativo.
3. Eliminados el archivo zip: `!rm F-F_Research_Data_Factors_CSV.zip`.



In [54]:
!wget http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/F-F_Research_Data_Factors_CSV.zip
!unzip -a F-F_Research_Data_Factors_CSV.zip
!rm F-F_Research_Data_Factors_CSV.zip

--2023-11-05 20:25:07--  http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/F-F_Research_Data_Factors_CSV.zip
Resolving mba.tuck.dartmouth.edu (mba.tuck.dartmouth.edu)... 129.170.136.60
Connecting to mba.tuck.dartmouth.edu (mba.tuck.dartmouth.edu)|129.170.136.60|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12773 (12K) [application/x-zip-compressed]
Saving to: 'F-F_Research_Data_Factors_CSV.zip'

F-F_Research_Data_F 100%[===================>]  12.47K  --.-KB/s    in 0s      

2023-11-05 20:25:08 (73.4 MB/s) - 'F-F_Research_Data_Factors_CSV.zip' saved [12773/12773]

Archive:  F-F_Research_Data_Factors_CSV.zip
  inflating: F-F_Research_Data_Factors.CSV  [binary]


Recuperamos el archivo para eso utilizamos la función de pandas `read_csv`. Las tres primeras filas tienen información que no nos interesa, por tanto las eliminamos con `skiprows=3`

In [74]:
factor_df = pd.read_csv('F-F_Research_Data_Factors.csv', 
                         skiprows=3)


In [75]:
factor_df.head()

,Unnamed: 0,Mkt-RF,SMB,HML,RF
0,192607,2.96,-2.56,-2.43,0.22
1,192608,2.64,-1.17,3.82,0.25
2,192609,0.36,-1.40,0.13,0.23
3,192610,-3.24,-0.09,0.70,0.32
4,192611,2.53,-0.10,-0.51,0.31


In [76]:
RISKY_ASSET = 'AMZN'
MARKET_BENCHMARK = '^GSPC'
START_DATE = '2014-01-01'
END_DATE = '2018-12-31'

Renombramos las columnas

In [77]:
factor_df.columns = ['date', 'mkt', 'smb', 'hml', 'rf']
factor_df.head()

,date,mkt,smb,hml,rf
0,192607,2.96,-2.56,-2.43,0.22
1,192608,2.64,-1.17,3.82,0.25
2,192609,0.36,-1.40,0.13,0.23
3,192610,-3.24,-0.09,0.70,0.32
4,192611,2.53,-0.10,-0.51,0.31


Nos aseguramos que la información contenida en la columna `date`sea una variable de tipo string. Es decir, una cadena alfanumérica.

In [78]:
factor_df['date'] = factor_df['date'].astype(str)
factor_df.head()

,date,mkt,smb,hml,rf
0,192607,2.96,-2.56,-2.43,0.22
1,192608,2.64,-1.17,3.82,0.25
2,192609,0.36,-1.40,0.13,0.23
3,192610,-3.24,-0.09,0.70,0.32
4,192611,2.53,-0.10,-0.51,0.31


Dado que el formato de la fecha es AAAAMM, eliminamos todas las filas que tengan alguna fecha de más de seis dígitos. 

In [79]:
factor_df = factor_df[factor_df['date'].str.len() == 6]
factor_df.head()

,date,mkt,smb,hml,rf
0,192607,2.96,-2.56,-2.43,0.22
1,192608,2.64,-1.17,3.82,0.25
2,192609,0.36,-1.40,0.13,0.23
3,192610,-3.24,-0.09,0.70,0.32
4,192611,2.53,-0.10,-0.51,0.31


In [72]:
factor_df['date'] = pd.to_datetime(factor_df['date'],format='%Y%m', errors='coerce').dt.strftime("%Y-%m")
factor_df.head()                     

,date,mkt,smb,hml,rf
0,1926-07,2.96,-2.56,-2.43,0.22
1,1926-08,2.64,-1.17,3.82,0.25
2,1926-09,0.36,-1.40,0.13,0.23
3,1926-10,-3.24,-0.09,0.70,0.32
4,1926-11,2.53,-0.10,-0.51,0.31


In [58]:
STR_TO_MATCH = ' Annual Factors: January-December '
indices = factor_df.iloc[:, 0] == STR_TO_MATCH
start_of_annual = factor_df[indices].index[0]
factor_df.head()

,Unnamed: 0,Mkt-RF,SMB,HML,RF
0,192607,2.96,-2.56,-2.43,0.22
1,192608,2.64,-1.17,3.82,0.25
2,192609,0.36,-1.40,0.13,0.23
3,192610,-3.24,-0.09,0.70,0.32
4,192611,2.53,-0.10,-0.51,0.31
